In [1]:
# from mobile_net import get_model
import librosa
import tensorflow as tf
import glob
import numpy as np
import random
import wave
import os
import pickle
# import noisereduce as nr
import requests

2024-10-30 08:01:39.676154: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-30 08:01:41.341414: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-30 08:01:41.932594: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-30 08:01:42.093745: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-30 08:01:43.300432: I tensorflow/core/platform/cpu_feature_guar

In [2]:
gpus = tf.config.list_physical_devices('GPU')
gpu = gpus[0]
tf.config.experimental.set_memory_growth(gpu, True)
tf.config.list_logical_devices('GPU')

2024-10-30 08:01:50.771579: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43629 MB memory:  -> device: 0, name: NVIDIA L40S, pci bus id: 0000:8b:00.0, compute capability: 8.9


[LogicalDevice(name='/device:GPU:0', device_type='GPU')]

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
noise_path = '/shareddrive/working/data_code/data/neg_data/_background_noise_/chunks/2-5s_chunks/*'
environment_path = '/shareddrive/working/data_code/data/neg_data/envornment/chunks/2-5s_chunks/*'
word_path = '/shareddrive/working/data_code/data/neg_data/spcmd/all_words/2-5s_chunks/*'
recording_path = '/shareddrive/working/data_code/data/neg_data/internet_recordings/chunks/2-5s_chunks/2-5s_chunks/*'
aug_recording_path = '/shareddrive/working/data_code/data/neg_data/internet_recordings/chunks/2-5s_chunks/2-5s_chunks_aug/*'

In [7]:
word_files = glob.glob(word_path)
environment_files = glob.glob(environment_path)
noise_files = glob.glob(noise_path)
recording_files = glob.glob(recording_path)
aug_recording_files = glob.glob(aug_recording_path)
len(environment_files),len(noise_files),len(word_files),len(recording_files),len(aug_recording_files)

(515, 264, 64721, 8649, 16636)

In [8]:
# def get_duration(audio_path):
#     with wave.open(audio_path, 'rb') as wav_file:
#         sample_rate = wav_file.getframerate()
#         num_frames = wav_file.getnframes()
#         duration = num_frames / sample_rate
#     return duration

In [8]:
limit = 1.5
# keyword_path = '/shareddrive/working/data_code/data/adele/augmented/*'
# keyword_folders = glob.glob(keyword_path)
# keyword_files = [j for i in keyword_folders for j in glob.glob(f'{i}/*')]
# keyword_files = [j for i in keyword_folders for j in glob.glob(f'{i}/*') if get_duration(j) <= limit]
keyword_path = '/shareddrive/working/data_code/data/hilfe_hilfe/augmented/*'
keyword_folders = glob.glob(keyword_path)
keyword_files = [j for i in keyword_folders for j in glob.glob(f'{i}/*')]
# keyword_files = [j for i in keyword_folders for j in glob.glob(f'{i}/*') if get_duration(j) <= limit]

In [9]:
len(keyword_files)

28500

In [46]:
# DEEPGRAM_API_KEY = '4207b8a639744fbdbe634616684bf7d67c8791e2'
# url = 'https://api.deepgram.com/v1/listen'
# headers = {
#     'Authorization': f'Token {DEEPGRAM_API_KEY}',
#     'Content-Type': 'audio/wav'
# }
# params = {
#     'language': 'de',  # Specify the language as German
#     'tier': 'enhanced',  # Use the enhanced tier for better accuracy
#     'model': 'general'  # Specify the model
# }
# ad_files = list()
# for i in adele_files:
#     # Read the audio file
#     with open(i, 'rb') as audio_file:
#         response = requests.post(url, headers=headers, params=params, data=audio_file)

#     if response.status_code == 200:
#         if 'adele' in response.json()['results']['channels'][0]['alternatives'][0]['transcript']:
#             ad_files.append(i)
#     else:
#         print("Error:", response.json())
# len(ad_files)

In [15]:
no_of_files = 1109+660
file_path_and_labels = list()
# file_path_and_labels.extend([(i,1) for i in random.sample(adele_files,no_of_files)])
file_path_and_labels.extend([(i,1) for i in keyword_files])
# avg_files = (len(adele_files) + len(hilfe_files)) // 2
avg_files = int(len(keyword_files)/1.5)
file_path_and_labels.extend([(i,0) for i in recording_files])
file_path_and_labels.extend([(i,0) for i in aug_recording_path])
file_path_and_labels.extend([(i,0) for i in random.sample(word_files,avg_files)])
file_path_and_labels.extend([(i,0) for i in noise_files])
file_path_and_labels.extend([(i,0) for i in environment_files])
random.shuffle(file_path_and_labels)
# avg_files
len(file_path_and_labels),len(keyword_files),len(file_path_and_labels)-len(keyword_files),

(57029, 28500, 28529)

In [16]:
sr = 16000
max_seconds = 2.5
pad_or_trunc = lambda a,i : a[0:i] if len(a) > i else a if len(a) == i else np.pad(a,(0, (i-len(a))))

def process_data(y,sr,max_seconds):
    y = pad_or_trunc(y,int(sr*max_seconds))
    features = librosa.feature.melspectrogram(y=y,sr=sr,n_fft=1024)
    return features
    
def get_processed_data(audio_file):
    y,_ = librosa.load(audio_file,sr=sr)
    # y = nr.reduce_noise(y=y, sr=sr,n_fft=1024)
    # y[np.isnan(y)] = 0
    features = process_data(y,sr,max_seconds)
    return features
try:
    with open('f_and_ls.pickle','rb') as f:
        features_and_labels = pickle.load(f)
except:
    features_and_labels = list()
    for i,j in file_path_and_labels:
        try:
            features = get_processed_data(i)
            features_and_labels.append((features,j))
        except Exception as e: 
            print(i)
            print(e)
        # features = get_processed_data(i)
        # features_and_labels.append(features,j)
    # with open('f_and_l.pickle','wb') as f:
    #     pickle.dump(features_and_labels,f)
finally:
    print(features_and_labels[0][0].shape)

/tmp/ipykernel_51/2894920758.py:11: UserWarning: PySoundFile failed. Trying audioread instead.
  y,_ = librosa.load(audio_file,sr=sr)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


i
[Errno 2] No such file or directory: 'i'
r
[Errno 2] No such file or directory: 'r'
s
[Errno 2] No such file or directory: 's'
d
[Errno 2] No such file or directory: 'd'
/
[Errno 21] Is a directory: '/'
a
[Errno 2] No such file or directory: 'a'
g
[Errno 2] No such file or directory: 'g'
e
[Errno 2] No such file or directory: 'e'
_
[Errno 2] No such file or directory: '_'
r
[Errno 2] No such file or directory: 'r'
n
[Errno 2] No such file or directory: 'n'
o
[Errno 2] No such file or directory: 'o'
t
[Errno 2] No such file or directory: 't'
t
[Errno 2] No such file or directory: 't'
r
[Errno 2] No such file or directory: 'r'
u
[Errno 2] No such file or directory: 'u'
-
[Errno 2] No such file or directory: '-'
d
[Errno 2] No such file or directory: 'd'
/
[Errno 21] Is a directory: '/'
n
[Errno 2] No such file or directory: 'n'
n
[Errno 2] No such file or directory: 'n'
g
[Errno 2] No such file or directory: 'g'
r
[Errno 2] No such file or directory: 'r'
h
[Errno 2] No such file or dir

In [17]:


class DataSequenceRaw(tf.keras.utils.Sequence):

    def __init__(self,data,batch_size):
        self.data_size = len(data)
        audios,labels = zip(*data)
        self.X = np.array(audios)
        self.Y = np.array(labels)
        self.batch_size = batch_size

        
    def __len__(self):
        return int(np.ceil(self.data_size / self.batch_size))

    def __getitem__(self,idx):
        s = idx * self.batch_size
        e = (idx + 1) * self.batch_size
        X = self.X[s:e]
        Y = self.Y[s:e]
        return X,Y


In [18]:
def _get_train_val_size(total_examples,train_percent=70):
    
    train_ratio = round(train_percent)/100
    remaining_percent = 100 - train_percent
    test_percent = round(remaining_percent/3)*2
    val_percent = remaining_percent - test_percent
    val_ratio = val_percent/100
    train_size = round(total_examples * train_ratio)
    val_size = round(total_examples * val_ratio)
    return train_size,val_size

def get_data_raw(data,train_percent=70,batch_size=32):

    train_size,val_size = _get_train_val_size(len(data),train_percent)

    train_examples = data[:train_size]
    val_examples = data[train_size:train_size+val_size]
    test_examples = data[train_size+val_size:]

    train = DataSequenceRaw(train_examples,batch_size=batch_size)
    test = DataSequenceRaw(test_examples,batch_size=1)
    val = DataSequenceRaw(val_examples,batch_size=batch_size)

    return train,test,val

train,test,val = get_data_raw(features_and_labels,train_percent=80)

In [19]:
shape = train[0][0][0].shape
input_shape = [*shape,1]

In [20]:
def get_model(
        input_shape,
        output_neurons=1,
        output_activation='sigmoid',
        loss=tf.keras.losses.binary_crossentropy,
        lr=0.0001
):
    _input = tf.keras.layers.Input(shape=input_shape)
    x = tf.keras.layers.Conv2D(512,kernel_size=3,padding='valid',activation='relu')(_input)
    x = tf.keras.layers.Conv2D(256,kernel_size=3,padding='valid',activation='relu')(x)
    x = tf.keras.layers.MaxPool2D((2,2))(x)
    x = tf.keras.layers.Conv2D(128,kernel_size=3,padding='valid',activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Conv2D(128,kernel_size=3,padding='valid',activation='relu')(x)
    x = tf.keras.layers.MaxPool2D((2,2))(x)
    x = tf.keras.layers.Conv2D(64,kernel_size=3,padding='valid',activation='relu')(x)
    x = tf.keras.layers.MaxPool2D((2,2))(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1024,activation='relu')(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(1024,activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(1024,activation='relu')(x)
    x = tf.keras.layers.Dropout(0.7)(x)
    x = tf.keras.layers.Dense(1024,activation='relu')(x)
    x = tf.keras.layers.Dense(10,activation='relu')(x)
    outputs = tf.keras.layers.Dense(output_neurons,activation=output_activation,kernel_regularizer=tf.keras.regularizers.L2(l2=0.01))(x)
    model = tf.keras.Model(inputs=_input,outputs=outputs)

    model.compile(
        loss=loss,
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        metrics=['accuracy'],
    )

    return model

model = get_model(
        input_shape=input_shape,
        lr=0.001
)



In [21]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy',factor=0.1,patience=3,mode='max')
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=5,mode='max',restore_best_weights=True,start_from_epoch=10)
with tf.device('/gpu'):
    history = model.fit(train,epochs=100,validation_data=val,verbose=1,callbacks=[reduce_lr,early_stopping])

Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1730277920.475277     160 service.cc:146] XLA service 0x7fdfdc004450 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730277920.475310     160 service.cc:154]   StreamExecutor device (0): NVIDIA L40S, Compute Capability 8.9
2024-10-30 08:45:20.608086: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-30 08:45:21.146752: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8906
2024-10-30 08:45:22.257379: I external

   3/1424 ━━━━━━━━━━━━━━━━━━━━ 1:05 46ms/step - accuracy: 0.4549 - loss: 3.1955  

I0000 00:00:1730277931.672699     160 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 636/1424 ━━━━━━━━━━━━━━━━━━━━ 40s 51ms/step - accuracy: 0.7874 - loss: 0.5629

2024-10-30 08:46:04.228758: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'input_reduce_select_fusion_4', 72 bytes spill stores, 72 bytes spill loads



1424/1424 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.8471 - loss: 0.4605

2024-10-30 08:46:40.183329: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_102', 132 bytes spill stores, 132 bytes spill loads

2024-10-30 08:46:45.379701: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_102', 32 bytes spill stores, 32 bytes spill loads

2024-10-30 08:46:45.445107: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_102', 12 bytes spill stores, 12 bytes spill loads

2024-10-30 08:46:45.563395: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_102', 200 bytes spill stores, 200 bytes spill loads

2024-10-30 08:46:46.056445: I external/local_xla/xla/stream_

1424/1424 ━━━━━━━━━━━━━━━━━━━━ 90s 54ms/step - accuracy: 0.8471 - loss: 0.4604 - val_accuracy: 0.9848 - val_loss: 0.1749 - learning_rate: 0.0010
Epoch 2/100
1424/1424 ━━━━━━━━━━━━━━━━━━━━ 64s 45ms/step - accuracy: 0.9713 - loss: 0.1841 - val_accuracy: 0.4880 - val_loss: 1.0790 - learning_rate: 0.0010
Epoch 3/100
1424/1424 ━━━━━━━━━━━━━━━━━━━━ 65s 46ms/step - accuracy: 0.4993 - loss: 0.8917 - val_accuracy: 0.4880 - val_loss: 0.7123 - learning_rate: 0.0010
Epoch 4/100
1424/1424 ━━━━━━━━━━━━━━━━━━━━ 65s 46ms/step - accuracy: 0.5004 - loss: 0.7005 - val_accuracy: 0.4880 - val_loss: 0.6939 - learning_rate: 0.0010
Epoch 5/100
1424/1424 ━━━━━━━━━━━━━━━━━━━━ 65s 46ms/step - accuracy: 0.5025 - loss: 0.6933 - val_accuracy: 0.4880 - val_loss: 0.6937 - learning_rate: 1.0000e-04
Epoch 6/100
1424/1424 ━━━━━━━━━━━━━━━━━━━━ 65s 46ms/step - accuracy: 0.5047 - loss: 0.6931 - val_accuracy: 0.4880 - val_loss: 0.6935 - learning_rate: 1.0000e-04
Epoch 7/100
1424/1424 ━━━━━━━━━━━━━━━━━━━━ 65s 46ms/step - acc

In [22]:
model.evaluate(test)

7970/7970 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - accuracy: 0.4897 - loss: 0.6934


[0.6933478713035583, 0.4905897080898285]

In [23]:
folder_path = '/shareddrive/working/model_code/models/custom_model_4/trail_8'
os.makedirs(folder_path,exist_ok=True)
model_path = f'{folder_path}/16k_melspec-nfft-1024_h_cnn_dense_model.keras'
model.save(model_path)